In [1]:
import pandas as pd
import math
from statistics import mode 
from sklearn.model_selection import train_test_split
import numpy as np
import math
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("C://Users//Trivikram//ML_Projects//Data//Adult.data",header=None,names=['Age','Workplace','fnlwgt','education','education num','marital-stauts','occupation','relationship','race','sex','capital gain','capital loss','hours per week','native country','Salary'])
df

,Age,Workplace,fnlwgt,education,education num,marital-stauts,occupation,relationship,race,sex,capital gain,capital loss,hours per week,native country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
def removeQuestionMark(outCol,inpCol,X,y):
    for i in inpCol:
        repl=X[i].value_counts().keys().tolist()[0]
        X[i]=X[i].replace(to_replace=' ?',value=repl)
    rep=y[outCol[0]].value_counts().keys().tolist()[0]
    y[outCol[0]]=y[outCol[0]].replace(to_replace=' ?',value=rep)
    return[X,y]

In [4]:
y=df[['Salary']]
X=df[['Age','fnlwgt','education num','capital gain','capital loss','hours per week','Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']]
inpCol=list(X.columns)
outCol=list(y.columns)
(X,y)=removeQuestionMark(outCol,inpCol,X,y)
categCol=['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']
contCol=['Age','fnlwgt','education num','capital gain','capital loss','hours per week']

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
scaler = StandardScaler()
X=pd.DataFrame(data=scaler.fit_transform(df[contCol]),columns=contCol)
X

,Age,fnlwgt,education num,capital gain,capital loss,hours per week
0,0.030671,-1.063611,1.134739,0.148453,-0.21666,-0.035429
1,0.837109,-1.008707,1.134739,-0.145920,-0.21666,-2.222153
2,-0.042642,0.245079,-0.420060,-0.145920,-0.21666,-0.035429
3,1.057047,0.425801,-1.197459,-0.145920,-0.21666,-0.035429
4,-0.775768,1.408176,1.134739,-0.145920,-0.21666,-0.035429
...,...,...,...,...,...,...
32556,-0.849080,0.639741,0.746039,-0.145920,-0.21666,-0.197409
32557,0.103983,-0.335433,-0.420060,-0.145920,-0.21666,-0.035429
32558,1.423610,-0.358777,-0.420060,-0.145920,-0.21666,-0.035429
32559,-1.215643,0.110960,-0.420060,-0.145920,-0.21666,-1.655225


In [96]:
def LogisticBatchDescent(X,y,weights,learningRate):
    X['Column for weight 0']=1
    predictionProba=np.zeros(X.shape[0])
    previousNLL=np.inf
    NLL=0
    previousWeights=weights
    flag=0
    while previousNLL-NLL>=0.0000001:
        if flag==0:
            pass
        else:
            previousNLL=NLL
        predictionProba=X.apply(lambda x:1/(1+np.exp(-np.sum(np.multiply(x,weights)))),axis=1).tolist()
        NLL=-np.sum(np.nan_to_num(list(map(lambda y, p : ((1-y)*np.log(1-p)+y*np.log(p)), y, predictionProba))))
        print("Negative Log Likelihood is "+str(NLL))
        print("The weights are "+str(weights))
        gradientDescentNLL=X.apply(lambda x:-np.sum(x*(np.subtract(y,predictionProba)),axis=0)).values
        previousWeights=weights
        weights=weights-learningRate*gradientDescentNLL
        flag=flag+1
    return previousWeights


In [7]:
indexSalaryHigh=df.loc[df['Salary'] ==' >50K'].index
y_proba=np.zeros(df.shape[0])
y_proba[indexSalaryHigh]=1
X_train, X_test, y_train, y_test = train_test_split( X, y_proba, test_size=0.3, random_state=42)
weights=Constants=np.random.rand(len(X_train.columns)+1)
learningRate=float(input("Enter the learning Rate you intend to use "))

Enter the learning Rate you intend to use 0.01


In [125]:
weights

array([0.60869471, 0.54113371, 0.63093412, 0.87558054, 0.44387586,
       0.77527701, 0.77142675])

In [126]:
weights=LogisticBatchDescent(X_train,y_train,weights,learningRate)

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Negative Log Likelihood is 19398.659676435793
The weights are [0.60869471 0.54113371 0.63093412 0.87558054 0.44387586 0.77527701
 0.77142675]
Negative Log Likelihood is 12984.367422530006
The weights are [ 0.56449609  0.37140796  0.64852247  0.98771317  0.43461308  0.66384296
 -0.09006249]
Negative Log Likelihood is 10689.967021676268
The weights are [ 0.51853556  0.24662467  0.62892242  1.06054784  0.38222374  0.56480157
 -0.5943468 ]
Negative Log Likelihood is 9914.104969445376
The weights are [ 0.49704471  0.1708402   0.62496386  1.11160252  0.32723229  0.50628599
 -0.8819101 ]
Negative Log Likelihood is 9639.620801693516
The weights are [ 0.49404123  0.12620718  0.63810835  1.15113609  0.28953698  0.47885984
 -1.04952751]
Negative Log Likelihood is 9528.423720802046
The weights are [ 0.50062471  0.09929787  0.65964936  1.18425834  0.26893227  0.46911733
 -1.1518423 ]
Negative Log Likelihood is 9474.259819302266
The weights are [ 0.51103999  0.08271682  0.683172    1.21351844  0.259

Negative Log Likelihood is 9307.478548824918
The weights are [ 0.59555466  0.05993809  0.82694892  1.89152939  0.26664062  0.51832768
 -1.38162128]
Negative Log Likelihood is 9306.922303391622
The weights are [ 0.59551071  0.05993645  0.82698613  1.89899426  0.26675328  0.51833688
 -1.38100117]
Negative Log Likelihood is 9306.385678679804
The weights are [ 0.5954677   0.05993492  0.82702313  1.90632603  0.26686358  0.51834593
 -1.3803908 ]
Negative Log Likelihood is 9305.867893632887
The weights are [ 0.59542566  0.0599335   0.82705996  1.91352771  0.26697159  0.51835488
 -1.3797901 ]
Negative Log Likelihood is 9305.368203081387
The weights are [ 0.59538462  0.05993219  0.82709667  1.9206022   0.26707737  0.51836376
 -1.37919896]
Negative Log Likelihood is 9304.885895819767
The weights are [ 0.59534459  0.05993099  0.82713329  1.92755234  0.267181    0.51837261
 -1.37861728]
Negative Log Likelihood is 9304.420292803492
The weights are [ 0.59530556  0.05992991  0.82716984  1.93438086  0

Negative Log Likelihood is 9293.294753815588
The weights are [ 0.59425143  0.05996122  0.82881093  2.16415858  0.27055229  0.51882611
 -1.35846962]
Negative Log Likelihood is 9293.205321630972
The weights are [ 0.59424116  0.05996249  0.82883752  2.16714888  0.2705931   0.51883395
 -1.35821153]
Negative Log Likelihood is 9293.118560988874
The weights are [ 0.59423112  0.05996375  0.82886384  2.17009416  0.27063325  0.51884172
 -1.35795724]
Negative Log Likelihood is 9293.034387532758
The weights are [ 0.59422131  0.05996501  0.82888988  2.17299516  0.27067276  0.51884942
 -1.3577067 ]
Negative Log Likelihood is 9292.952719775001
The weights are [ 0.59421172  0.05996628  0.82891564  2.17585263  0.27071164  0.51885704
 -1.35745984]
Negative Log Likelihood is 9292.873478989517
The weights are [ 0.59420235  0.05996754  0.82894112  2.1786673   0.2707499   0.5188646
 -1.3572166 ]
Negative Log Likelihood is 9292.796589109654
The weights are [ 0.59419318  0.0599688   0.82896633  2.18143988  0.

Negative Log Likelihood is 9290.815197741922
The weights are [ 0.59391387  0.06002314  0.82991534  2.27883493  0.27208777  0.51916097
 -1.34851199]
Negative Log Likelihood is 9290.797773344393
The weights are [ 0.59391063  0.060024    0.82992901  2.28015439  0.27210509  0.51916522
 -1.3483967 ]
Negative Log Likelihood is 9290.780835998117
The weights are [ 0.59390744  0.06002485  0.8299425   2.28145528  0.27212216  0.51916941
 -1.34828301]
Negative Log Likelihood is 9290.764371732506
The weights are [ 0.59390432  0.0600257   0.82995583  2.28273786  0.27213899  0.51917355
 -1.3481709 ]
Negative Log Likelihood is 9290.748366992473
The weights are [ 0.59390126  0.06002654  0.82996898  2.28400242  0.27215557  0.51917764
 -1.34806036]
Negative Log Likelihood is 9290.732808626204
The weights are [ 0.59389825  0.06002736  0.82998197  2.28524921  0.27217191  0.51918169
 -1.34795135]
Negative Log Likelihood is 9290.717683871313
The weights are [ 0.59389529  0.06002818  0.8299948   2.2864785   0

...

Negative Log Likelihood is 9290.182094867683
The weights are [ 0.59372078  0.06009039  0.83091194  2.37049812  0.27327319  0.51947546
 -1.34046291]
Negative Log Likelihood is 9290.182056376378
The weights are [ 0.59372067  0.06009044  0.83091264  2.37056012  0.27327398  0.51947569
 -1.34045744]
Negative Log Likelihood is 9290.182018906375
The weights are [ 0.59372056  0.06009049  0.83091334  2.37062129  0.27327476  0.51947591
 -1.34045204]
Negative Log Likelihood is 9290.181982431091
The weights are [ 0.59372046  0.06009054  0.83091403  2.37068164  0.27327553  0.51947613
 -1.34044671]
Negative Log Likelihood is 9290.181946924387
The weights are [ 0.59372035  0.06009059  0.8309147   2.37074119  0.27327629  0.51947635
 -1.34044146]
Negative Log Likelihood is 9290.181912359416
The weights are [ 0.59372025  0.06009064  0.83091537  2.37079994  0.27327703  0.51947656
 -1.34043627]
Negative Log Likelihood is 9290.181878711906
The weights are [ 0.59372015  0.06009068  0.83091603  2.37085791  0

Negative Log Likelihood is 9290.180965915883
The weights are [ 0.59371647  0.06009238  0.8309399   2.37295402  0.27330446  0.51948442
 -1.34024614]
Negative Log Likelihood is 9290.18095737314
The weights are [ 0.59371642  0.06009241  0.83094024  2.37298323  0.27330483  0.51948452
 -1.34024357]
Negative Log Likelihood is 9290.180949057176
The weights are [ 0.59371637  0.06009243  0.83094056  2.37301204  0.2733052   0.51948463
 -1.34024102]
Negative Log Likelihood is 9290.180940961345
The weights are [ 0.59371632  0.06009246  0.83094089  2.37304048  0.27330556  0.51948473
 -1.34023851]
Negative Log Likelihood is 9290.180933080315
The weights are [ 0.59371627  0.06009248  0.83094121  2.37306853  0.27330592  0.51948483
 -1.34023604]
Negative Log Likelihood is 9290.180925408236
The weights are [ 0.59371622  0.0600925   0.83094152  2.37309621  0.27330627  0.51948493
 -1.34023359]
Negative Log Likelihood is 9290.180917939064
The weights are [ 0.59371617  0.06009252  0.83094184  2.37312352  0.

Negative Log Likelihood is 9290.180715203236
The weights are [ 0.59371445  0.06009333  0.8309531   2.3741114   0.27331919  0.51948864
 -1.34014397]
Negative Log Likelihood is 9290.18071330467
The weights are [ 0.59371443  0.06009334  0.83095326  2.37412517  0.27331936  0.51948869
 -1.34014275]
Negative Log Likelihood is 9290.180711456276
The weights are [ 0.5937144   0.06009335  0.83095342  2.37413876  0.27331953  0.51948874
 -1.34014155]
Negative Log Likelihood is 9290.180709656901
The weights are [ 0.59371438  0.06009336  0.83095357  2.37415217  0.27331971  0.51948879
 -1.34014037]
Negative Log Likelihood is 9290.180707904874
The weights are [ 0.59371436  0.06009337  0.83095372  2.37416539  0.27331987  0.51948884
 -1.3401392 ]
Negative Log Likelihood is 9290.180706199848
The weights are [ 0.59371433  0.06009338  0.83095387  2.37417844  0.27332004  0.51948889
 -1.34013805]
Negative Log Likelihood is 9290.180704539494
The weights are [ 0.59371431  0.06009339  0.83095402  2.37419132  0.

Negative Log Likelihood is 9290.180659462714
The weights are [ 0.5937135   0.06009377  0.83095933  2.37465714  0.27332613  0.51949064
 -1.34009579]
Negative Log Likelihood is 9290.180659040427
The weights are [ 0.59371349  0.06009378  0.83095941  2.37466364  0.27332621  0.51949066
 -1.34009521]
Negative Log Likelihood is 9290.180658629488
The weights are [ 0.59371348  0.06009378  0.83095948  2.37467004  0.27332629  0.51949069
 -1.34009465]
Negative Log Likelihood is 9290.180658229045
The weights are [ 0.59371347  0.06009379  0.83095955  2.37467637  0.27332637  0.51949071
 -1.34009409]
Negative Log Likelihood is 9290.18065783929
The weights are [ 0.59371346  0.06009379  0.83095962  2.3746826   0.27332645  0.51949073
 -1.34009354]
Negative Log Likelihood is 9290.180657459929
The weights are [ 0.59371345  0.0600938   0.83095969  2.37468876  0.27332653  0.51949076
 -1.340093  ]
Negative Log Likelihood is 9290.180657090968
The weights are [ 0.59371344  0.0600938   0.83095976  2.37469483  0.

array([ 0.59371306,  0.06009398,  0.83096224,  2.37491143,  0.27332936,
        0.51949157, -1.34007334])

weights=[0.59371306,
 0.06009398,
 0.83096224,
 2.37491143,
 0.27332936,
 0.51949157,
 -1.34007334]

In [32]:
def prediction(X,y,weights):
    X['Column for weight 0']=1
    predictionProba=X.apply(lambda x:1/(1+np.exp(-np.sum(np.multiply(x,weights)))),axis=1).tolist()
    convertProbToLabel = lambda prob: 1 if prob>=0.5  else  0
    y_predictions = np.array([convertProbToLabel(i) for i in predictionProba])
    return y_predictions

In [33]:
y_predictions=prediction(X_test,y_test,weights)

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
len(y_predictions)

9769

In [36]:
y_predictions

array([0, 0, 0, ..., 0, 0, 1])

In [37]:
accuracy_score(y_predictions,y_test)

0.8173815129491248

In [64]:
admission=pd.read_csv("C://Users//Trivikram//ML_Projects//Data//Admission.csv")
admission.columns=['Serial No', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP','LOR ', 'CGPA', 'Research','Chance of Admit']
admission

,Serial No,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [82]:
X=admission[[col for col in admission.columns if col!='Chance of Admit' and col!='Serial No']]
y=admission['Chance of Admit']

In [83]:
scaler = StandardScaler()
X=pd.DataFrame(data=scaler.fit_transform(X),columns=X.columns)
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,1.762107,1.746971,0.798829,1.093864,1.167321,1.764818,0.909112
1,0.627656,-0.067635,0.798829,0.596653,1.167321,0.455151,0.909112
2,-0.070467,-0.562528,-0.076600,-0.397769,0.052933,-1.005631,0.909112
3,0.453126,0.427257,-0.076600,0.099442,-1.061454,0.119339,0.909112
4,-0.244998,-0.727492,-0.952029,-1.392191,-0.504260,-0.653029,-1.099975
...,...,...,...,...,...,...,...
395,0.627656,0.427257,-0.076600,0.099442,0.052933,0.740592,0.909112
396,0.714922,-0.067635,-0.076600,-0.397769,0.052933,0.858126,0.909112
397,1.151249,1.417042,0.798829,1.591075,1.167321,1.429006,0.909112
398,-0.419528,-0.727492,-0.076600,0.099442,0.610127,0.304036,-1.099975


In [101]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)
Weights=Constants=np.random.rand(len(X_train.columns)+1)
learningRate=float(input("Enter the learning Rate you intend to use "))

Enter the learning Rate you intend to use 0.001


In [102]:
Weights=LogisticBatchDescent(X_train,y_train,Weights,learningRate)

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Negative Log Likelihood is 194.4813275786018
The weights are [0.53443039 0.07502758 0.14383841 0.07195247 0.14979446 0.96854164
 0.17411186 0.59314243]
Negative Log Likelihood is 182.98003923174616
The weights are [0.49088656 0.03639716 0.10467864 0.03363763 0.11404261 0.92292719
 0.14210741 0.63119428]
Negative Log Likelihood is 174.19900261577564
The weights are [4.52199607e-01 2.68633688e-03 7.07983097e-02 6.55065845e-04
 8.31267663e-02 8.82195931e-01 1.14445310e-01 6.65659243e-01]
Negative Log Likelihood is 167.80827768073675
The weights are [ 0.41861114 -0.02589461  0.04236555 -0.02679487  0.05723392  0.84665569
  0.09118414  0.69665057]
Negative Log Likelihood is 163.35625356395352
The weights are [ 0.39008382 -0.04941495  0.01925259 -0.048829    0.03626496  0.81631477
  0.07215853  0.72440757]
Negative Log Likelihood is 160.35875703326596
The weights are [ 0.36630097 -0.06821963  0.00104842 -0.06586641  0.01984717  0.79087763
  0.0569993   0.74926145]
Negative Log Likelihood is 

Negative Log Likelihood is 152.37848249820019
The weights are [ 1.98704383e-01 -6.73970836e-02  6.07717567e-04 -3.35801961e-02
  2.06943586e-02  5.79922335e-01  3.15602833e-02  1.06734869e+00]
Negative Log Likelihood is 152.36733174868198
The weights are [ 0.19750737 -0.06598762  0.00134843 -0.03264471  0.02118154  0.57786182
  0.03193128  1.06869149]
Negative Log Likelihood is 152.35663079544557
The weights are [ 0.1963327  -0.0645845   0.00207218 -0.03173672  0.02165248  0.57582625
  0.03229458  1.06996249]
Negative Log Likelihood is 152.34635049421487
The weights are [ 0.19517982 -0.06318781  0.00277945 -0.03085535  0.0221079   0.57381517
  0.03265032  1.07116528]
Negative Log Likelihood is 152.336464115291
The weights are [ 0.19404822 -0.06179765  0.00347072 -0.02999972  0.02254849  0.57182816
  0.03299862  1.07230328]
Negative Log Likelihood is 152.3269471231928
The weights are [ 0.19293738 -0.0604141   0.00414644 -0.02916902  0.02297491  0.56986481
  0.03333961  1.07337972]
Negat

Negative Log Likelihood is 152.08382448989533
The weights are [ 0.15702292 -0.00264901  0.02432117 -0.00781238  0.03473103  0.49746372
  0.04332942  1.08938663]
Negative Log Likelihood is 152.08110096678678
The weights are [ 0.15653288 -0.00163122  0.02457022 -0.00760599  0.0348913   0.49631567
  0.04344248  1.08939966]
Negative Log Likelihood is 152.07843198911294
The weights are [ 1.56050318e-01 -6.20856125e-04  2.48145785e-02 -7.40544412e-03
  3.50502586e-02  4.95179322e-01  4.35528772e-02  1.08940850e+00]
Negative Log Likelihood is 152.07581627437048
The weights are [ 1.55575087e-01  3.82088590e-04  2.50543540e-02 -7.21059106e-03
  3.52079702e-02  4.94054532e-01  4.36606776e-02  1.08941341e+00]
Negative Log Likelihood is 152.07325258116373
The weights are [ 0.15510706  0.00137764  0.02528963 -0.00702129  0.03536449  0.49294116
  0.04376593  1.08941461]
Negative Log Likelihood is 152.07073970719392
The weights are [ 0.15464611  0.00236583  0.02552051 -0.00683739  0.03551988  0.49183

Negative Log Likelihood is 151.99289075278503
The weights are [ 0.13889628  0.04177748  0.03275818 -0.00236518  0.04223557  0.44994749
  0.04670757  1.0876739 ]
Negative Log Likelihood is 151.9919009826629
The weights are [ 0.13866937  0.04244647  0.0328502  -0.00233257  0.04236572  0.44926476
  0.04673652  1.08763088]
Negative Log Likelihood is 151.99092922243932
The weights are [ 0.1384456   0.04310991  0.03294049 -0.00230154  0.04249554  0.44858844
  0.04676466  1.0875881 ]
Negative Log Likelihood is 151.98997512710395
The weights are [ 0.1382249   0.04376782  0.03302908 -0.00227207  0.04262503  0.44791846
  0.04679199  1.08754556]
Negative Log Likelihood is 151.98903835881157
The weights are [ 0.13800724  0.04442026  0.03311601 -0.0022441   0.04275419  0.44725476
  0.04681855  1.08750329]
Negative Log Likelihood is 151.98811858671303
The weights are [ 0.13779257  0.04506725  0.03320129 -0.0022176   0.04288303  0.44659727
  0.04684435  1.08746128]
Negative Log Likelihood is 151.9872

...

Negative Log Likelihood is 151.93694910212872
The weights are [ 0.1224045   0.11423985  0.03711778 -0.00663112  0.06332236  0.37750254
  0.04674214  1.0834564 ]
Negative Log Likelihood is 151.93694477149552
The weights are [ 0.12240288  0.11428222  0.03711712 -0.00663903  0.06334154  0.37745659
  0.04673999  1.0834544 ]
Negative Log Likelihood is 151.93694051296728
The weights are [ 0.1224013   0.11432422  0.03711646 -0.00664689  0.06336056  0.37741101
  0.04673785  1.08345241]
Negative Log Likelihood is 151.93693632533729
The weights are [ 0.12239977  0.11436585  0.03711581 -0.00665468  0.06337944  0.3773658
  0.04673572  1.08345044]
Negative Log Likelihood is 151.93693220741898
The weights are [ 0.12239828  0.11440712  0.03711516 -0.00666242  0.06339817  0.37732096
  0.0467336   1.08344849]
Negative Log Likelihood is 151.93692815804584
The weights are [ 0.12239684  0.11444802  0.03711451 -0.0066701   0.06341676  0.37727648
  0.0467315   1.08344655]
Negative Log Likelihood is 151.9369

Negative Log Likelihood is 151.9367994047678
The weights are [ 0.1223677   0.11598817  0.03708936 -0.00696629  0.06413083  0.37557618
  0.04664746  1.08337338]
Negative Log Likelihood is 151.93679756197895
The weights are [ 0.1223677   0.11601521  0.03708891 -0.00697162  0.06414365  0.37554581
  0.04664589  1.08337209]
Negative Log Likelihood is 151.93679574973837
The weights are [ 0.12236771  0.11604201  0.03708847 -0.0069769   0.06415635  0.37551568
  0.04664432  1.08337081]
Negative Log Likelihood is 151.93679396753714
The weights are [ 0.12236775  0.11606857  0.03708803 -0.00698215  0.06416896  0.37548579
  0.04664277  1.08336954]
Negative Log Likelihood is 151.9367922148748
The weights are [ 0.12236781  0.11609491  0.03708759 -0.00698735  0.06418147  0.37545614
  0.04664123  1.08336828]
Negative Log Likelihood is 151.93679049125942
The weights are [ 0.12236789  0.11612101  0.03708716 -0.00699251  0.06419388  0.37542673
  0.0466397   1.08336703]
Negative Log Likelihood is 151.93678

Negative Log Likelihood is 151.93673560490828
The weights are [ 0.12238728  0.11710393  0.03707129 -0.0071901   0.06466931  0.37430124
  0.0465789   1.08331967]
Negative Log Likelihood is 151.9367348178371
The weights are [ 0.12238795  0.11712119  0.03707103 -0.00719363  0.06467781  0.37428111
  0.04657778  1.08331883]
Negative Log Likelihood is 151.9367340437575
The weights are [ 0.12238863  0.11713829  0.03707076 -0.00719713  0.06468625  0.37426114
  0.04657665  1.083318  ]
Negative Log Likelihood is 151.936733282454
The weights are [ 0.12238932  0.11715525  0.0370705  -0.0072006   0.06469462  0.37424133
  0.04657554  1.08331717]
Negative Log Likelihood is 151.9367325337146
The weights are [ 0.12239002  0.11717206  0.03707025 -0.00720404  0.06470292  0.37422168
  0.04657443  1.08331635]
Negative Log Likelihood is 151.936731797331
The weights are [ 0.12239072  0.11718872  0.03706999 -0.00720746  0.06471116  0.37420219
  0.04657334  1.08331554]
Negative Log Likelihood is 151.9367310730

Negative Log Likelihood is 151.93670797591997
The weights are [ 0.12242902  0.1178272   0.03706096 -0.00733986  0.06503178  0.37344215
  0.04652918  1.08328408]
Negative Log Likelihood is 151.93670764406343
The weights are [ 0.1224299   0.11783812  0.03706082 -0.00734215  0.06503736  0.37342889
  0.04652838  1.08328354]
Negative Log Likelihood is 151.93670731766062
The weights are [ 0.12243078  0.11784895  0.03706069 -0.00734442  0.0650429   0.37341574
  0.04652759  1.083283  ]
Negative Log Likelihood is 151.9367069966214
The weights are [ 0.12243166  0.11785968  0.03706055 -0.00734668  0.06504839  0.37340269
  0.04652681  1.08328246]
Negative Log Likelihood is 151.93670668085733
The weights are [ 0.12243254  0.11787032  0.03706042 -0.00734891  0.06505383  0.37338974
  0.04652603  1.08328193]
Negative Log Likelihood is 151.93670637028117
The weights are [ 0.12243343  0.11788086  0.03706029 -0.00735113  0.06505924  0.3733769
  0.04652525  1.0832814 ]
Negative Log Likelihood is 151.93670

Negative Log Likelihood is 151.9366963078661
The weights are [ 0.12247481  0.11828497  0.03705592 -0.00743677  0.06526929  0.3728758
  0.04649426  1.08326092]
Negative Log Likelihood is 151.93669616740672
The weights are [ 0.12247567  0.11829188  0.03705586 -0.00743825  0.06527294  0.37286705
  0.04649371  1.08326056]
Negative Log Likelihood is 151.93669602924544
The weights are [ 0.12247652  0.11829873  0.0370558  -0.00743971  0.06527657  0.37285837
  0.04649316  1.08326021]
Negative Log Likelihood is 151.93669589334445
The weights are [ 0.12247738  0.11830553  0.03705574 -0.00744117  0.06528016  0.37284976
  0.04649261  1.08325986]
Negative Log Likelihood is 151.9366957596666
The weights are [ 0.12247823  0.11831226  0.03705568 -0.00744261  0.06528372  0.37284121
  0.04649206  1.08325951]
Negative Log Likelihood is 151.9366956281753
The weights are [ 0.12247908  0.11831893  0.03705563 -0.00744403  0.06528726  0.37283274
  0.04649152  1.08325917]
Negative Log Likelihood is 151.9366954

In [103]:
def prediction(X,y,weights):
    X['Column for weight 0']=1
    predictionProba=X.apply(lambda x:1/(1+np.exp(-np.sum(np.multiply(x,weights)))),axis=1).tolist()
    return predictionProba

In [104]:
y_predictions=prediction(X_test,y_test,Weights)

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [105]:
error=sum((y_predictions-y_test)**2)/len(y_test)

In [106]:
100*error

0.49460633680061666

In [107]:
y_predictions

[0.651696684881512,
 0.733472139419516,
 0.9025666980334653,
 0.8284163197480378,
 0.55922656814212,
 0.8988434052910129,
 0.49577162937688607,
 0.5177096962391355,
 0.6783539252840104,
 0.8470830858807246,
 0.7453541863180415,
 0.890378657963002,
 0.5354760707019358,
 0.8739003204183706,
 0.7194998504668929,
 0.7069329178802453,
 0.6802263744779079,
 0.4333084249549477,
 0.7068584170265889,
 0.9194056975680108,
 0.5802149218069744,
 0.6670849933265949,
 0.7357303892984309,
 0.5073807462687753,
 0.9091826534370595,
 0.8217849698455951,
 0.7081934873606185,
 0.5477121381249294,
 0.6914634215526259,
 0.8262367442210068,
 0.8218436005000738,
 0.9077745487841133,
 0.6529264044617907,
 0.474067520438741,
 0.6773790475214538,
 0.6781837678888917,
 0.7419666006176973,
 0.6576870043329966,
 0.6242492494899288,
 0.8733684867373767,
 0.7594991042139212,
 0.5950174197483746,
 0.7969081264406835,
 0.9149776160361757,
 0.8488140430401424,
 0.8401234934736961,
 0.9097647190796789,
 0.665516718269613

In [108]:
y_test

209    0.68
280    0.68
33     0.90
210    0.79
93     0.44
       ... 
60     0.48
79     0.46
285    0.93
305    0.74
281    0.80
Name: Chance of Admit, Length: 120, dtype: float64